In [4]:
import keras 
import numpy as np
import tensorflow as tf
import os as os
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.models import Sequential 
from keras.layers import Embedding
from keras.layers import Dense
from keras import models
from keras import layers
import matplotlib.pyplot as plt

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.4 MB/s eta 0:00:00


In [2]:
import keras_tuner

**Build Model**

In [5]:
def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='mse', metrics=['mae'],
    )
    return model


def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model


build_model(keras_tuner.HyperParameters())

**Keras Tuner**

In [6]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="mae",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="Regression",
)

**Data PreProcess**





In [7]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Thesis/Notebook Modules/MoveData1.csv')  # import data

In [ ]:
# drop unwanted columns and do random shuffle of rows. 
newdf=df.sample(frac=1)
newdf=newdf.reset_index(drop=True)
newdf = newdf.drop(columns=['OM1X1', 'OM1X2',
       'OM1X3', 'OM1X4', 'OM1X5', 'OM1X6', 'OM2X1', 'OM2X2', 'OM2X3', 'OM2X4',
       'OM2X5', 'OM2X6'])
newdf.head()

In [ ]:
#seperate label column from data
Label = pd.DataFrame(newdf['Label'])
newdf = newdf.drop(columns=['Label'])

In [ ]:
# seperate targets
Target = pd.DataFrame(newdf[['VM1X1', 'VM1X2','VM1X3', 'VM1X4', 'VM1X5', 'VM1X6', 'VM2X1', 'VM2X2', 'VM2X3', 'VM2X4','VM2X5', 'VM2X6']])
Input  = newdf.drop(columns=['VM1X1', 'VM1X2','VM1X3', 'VM1X4', 'VM1X5', 'VM1X6', 'VM2X1', 'VM2X2', 'VM2X3', 'VM2X4','VM2X5', 'VM2X6'])
Input.head()

In [ ]:
InputArray  = np.asarray(Input)
TargetArray = np.asarray(Target)
LabelArray  = np.asarray(Label) 
InputArray.shape

In [ ]:
# separate good bad
badcount = 26522
goodcount = InputArray.shape[0]-badcount

GoodInput  = np.zeros([goodcount,24])
GoodTarget = np.zeros([goodcount,12])

row = InputArray.shape[0]
i = 0
m = 0
while(i<row):
  if(LabelArray[i]==1):
    GoodInput[m] = InputArray[i]
    GoodTarget[m] = TargetArray[i]
    m=m+1
    i=i+1
  else:
    i=i+1

In [ ]:
X_good = GoodInput
Y_good = GoodTarget

In [ ]:
train_data = X_good[0:11739,:]
test_data = X_good[11740:23477, :]
train_targets = Y_good[0:11739,0]
test_targets = Y_good[11740:23477,0]

**Train Model**

In [ ]:
tuner.search(train_data, train_targets, epochs=6, validation_data=(test_data, test_targets))

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])

In [ ]:
# Apply K fold
k = 2
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
num_epochs = 500
all_mae_histories = []

In [ ]:
# call backs
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
                        [train_data[:i * num_val_samples],
                        train_data[(i + 1) * num_val_samples:]],
                        axis=0)
  partial_train_targets = np.concatenate(
                        [train_targets[:i * num_val_samples],
                        train_targets[(i + 1) * num_val_samples:]],
                        axis=0)
  #model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,
  validation_data=(val_data, val_targets),
  epochs=num_epochs, batch_size=1, verbose=0,callbacks=my_callbacks)
  mae_history = history.history['mae']
  all_mae_histories.append(mae_history)

In [ ]:
import pickle

In [ ]:
filename = 'Navigation_Train.sav'
pickle.dump(model, open(filename, 'wb'))